In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from MARBLE import plotting, utils, geometry, net

In [2]:
k = 15
n_clusters = 10
    
par = {'batch_size': 256, #batch size, this should be as large as possible
        'epochs': 3, #optimisation epochs
        'order': 2, #order of derivatives
        'n_lin_layers': 2,
        'hidden_channels': 16, #number of internal dimensions in MLP
        'out_channels': 4,
        }

In [3]:
def f1(x):
    return np.cos(x[:,[0]]) + np.sin(x[:,[1]])

def f2(x):
    return np.cos(x[:,[0]]) + np.sin(2*x[:,[1]])

def sample_cone(alpha, nr, ntheta):
    r = np.sqrt(np.linspace(0.5, 5, nr))
    theta = np.linspace(0, 2*np.pi, ntheta)
    r, theta = np.meshgrid(r, theta)
    X = r*np.cos(theta)
    Y = r*np.sin(theta)
    Z = -(alpha*r)**2
    
    return np.column_stack([X.flatten(), Y.flatten(), Z.flatten()])

In [4]:
n_steps = 3
alpha = np.linspace(1, 0.750, n_steps)
nr = 10
ntheta = 40
x_ = np.linspace(-np.pi, np.pi, nr)
y_ = np.linspace(-np.pi, np.pi, ntheta)
x_, y_ = np.meshgrid(x_, y_)
X = np.column_stack([x_.flatten(), y_.flatten()])
    
y = [f1(X) for i in range(n_steps)] + [f2(X) for i in range(n_steps)]
x = 2 * [sample_cone(a, nr, ntheta) for a in alpha]
    
ind, _ = geometry.furthest_point_sampling(x[0], stop_crit=0.1)
x = [x_[ind] for x_ in x]
y = [y_[ind] for y_ in y]

In [5]:
data = utils.construct_dataset(x, y, graph_type='cknn', k=k)